In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import deepRD

In [ ]:
def read_numpy_arrays_from_file(file_path):
    arrays = []
    with open(file_path, 'r') as file:
        contents = file.read()

    # Split the file content based on square brackets [ ]
    array_strings = contents.split('[')[1:]

    # Convert each array string back to a numpy array and store it
    for array_string in array_strings:
        array = np.fromstring(array_string.replace(']', ''), sep=' ')
        arrays.append(array)

    return arrays

In [ ]:
# Read files
simID = '0000'
nsims = 10000
localDataDirectory = os.environ.get('DATA') + 'openSystems/classicSmoluchowski_' + simID
file_path = localDataDirectory + '/classicSmol_nsims' + str(nsims) + '.xyz'
concentrationArrays = read_numpy_arrays_from_file(file_path)